In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Discriminator_hojun import Discriminator
from Generator_hojun import Generator, make_layers, get_activation
from utils.loaders import load_safari

import numpy as np
import os
import matplotlib.pyplot as plot
import json
import pickle as pkl
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
num_epochs = 500
batch_size = 64
learning_rate = 0.0004
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DATA_NAME='camel'
(x_train, y_train) = load_safari(DATA_NAME)
_x_train = np.transpose(x_train, (0,3,1,2))
_y_train=np.array(y_train)
x_train = torch.from_numpy(_x_train).to(device)
y_train= torch.from_numpy(_y_train).to(device)
input_data = [[x_item, y_item] for x_item, y_item in zip(x_train, y_train)]

In [3]:
discriminator = Discriminator(input_dim=(60,1,28,28), discriminator_conv_channels = [64,64,128,128],
    discriminator_conv_kernel_size = [5,5,5,5], discriminator_conv_strides = [2,2,2,1],
    discriminator_batch_norm_momentum = None, discriminator_activation = 'relu',
    discriminator_dropout_rate = 0.4).to(device)
generator=Generator(input_dim=100, generator_initial_dense_layer_size = (64, 7, 7)
        , generator_upsample = [2,2, 1, 1]
        , generator_conv_channels = [128,64, 64,1]
        , generator_conv_kernel_size = [5,5,5,5]
        , generator_conv_strides = [1,1, 1, 1]
        , generator_batch_norm_momentum = 0.9
        , generator_activation = 'relu').to(device)

In [7]:
A=torch.zeros([batch_size, 1, 28, 28]).to(device)

In [10]:
B=torch.zeros([32, 100]).to(device)

In [11]:
generator(B).shape

torch.Size([32, 1, 28, 28])

In [12]:
print(generator)

Generator(
  (generator_layer1): Sequential(
    (0): Linear(in_features=100, out_features=3136, bias=True)
    (1): BatchNorm1d(3136, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (generator_layer2): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (generator_layer3): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (generator_layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (generator_layer5): Se

In [4]:
camel_loader=DataLoader(input_data, batch_size=batch_size, shuffle=True)
# Loss and optimizer
discriminator_criterion = nn.BCELoss()
generator_criterion = nn.BCELoss()
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)


In [5]:
# Train the model

for epoch in range(num_epochs):
    for _, (x_batch, y_batch) in enumerate(camel_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        if len(x_batch)<batch_size:
            valid_discriminator = np.ones((len(x_batch),1))
            fake_discriminator = np.zeros((len(x_batch),1))
        else:
            valid_discriminator = np.ones((batch_size,1))
            fake_discriminator = np.zeros((batch_size,1))
        
        valid_discriminator=torch.Tensor(valid_discriminator).to(device)
        fake_discriminator=torch.Tensor(fake_discriminator).to(device)
        noise = np.random.normal(0, 1, (batch_size, 100))
        noise = torch.Tensor(noise).to(device)
        gen_x_batch = generator(noise)
        total_x_batch=torch.cat((gen_x_batch, x_batch), 0)
        predicted_labels=discriminator(total_x_batch)
        labels=torch.cat((fake_discriminator, valid_discriminator), 0)
        predicted_labels=predicted_labels.squeeze(1)
        labels=labels.squeeze(1)
        
        discriminator_loss = discriminator_criterion(predicted_labels, labels)
        # Backward and optimize
        discriminator_optimizer.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()
        
        
        valid_generator = torch.Tensor(np.ones((batch_size,1))).to(device).squeeze(1)
        noise_generator = torch.Tensor(np.random.normal(0, 1, (batch_size, 100))).to(device)
        output_generator = generator(noise_generator)
        output_generator = discriminator(output_generator).squeeze(1)
        generator_loss=generator_criterion(output_generator, valid_generator)
        generator_optimizer.zero_grad()
        generator_loss.backward()
        generator_optimizer.step()
        
    print("%d: Discriminator Loss=%.6f, Generator Loss=%.6f" % (
        epoch, discriminator_loss.item(), generator_loss.item()))
        

0: Discriminator Loss=0.000010, Generator Loss=1.526747
1: Discriminator Loss=0.000068, Generator Loss=0.986471
2: Discriminator Loss=0.000002, Generator Loss=1.095554
3: Discriminator Loss=0.000001, Generator Loss=1.523482
4: Discriminator Loss=0.000000, Generator Loss=2.201842
5: Discriminator Loss=0.000000, Generator Loss=2.298115
6: Discriminator Loss=0.000000, Generator Loss=2.458674
7: Discriminator Loss=0.000000, Generator Loss=3.348754
8: Discriminator Loss=0.000000, Generator Loss=3.481417
9: Discriminator Loss=0.000000, Generator Loss=4.488234
10: Discriminator Loss=0.000559, Generator Loss=11.218584
11: Discriminator Loss=0.000000, Generator Loss=1.330423
12: Discriminator Loss=0.000003, Generator Loss=1.462692
13: Discriminator Loss=0.000008, Generator Loss=2.193399
14: Discriminator Loss=0.000000, Generator Loss=3.083668
15: Discriminator Loss=0.000000, Generator Loss=0.991642
16: Discriminator Loss=0.000000, Generator Loss=2.514071
17: Discriminator Loss=0.000000, Generat

144: Discriminator Loss=0.000000, Generator Loss=1.503276
145: Discriminator Loss=0.000000, Generator Loss=1.384340
146: Discriminator Loss=0.000000, Generator Loss=1.513828
147: Discriminator Loss=0.000000, Generator Loss=1.958246
148: Discriminator Loss=0.000000, Generator Loss=1.638228
149: Discriminator Loss=0.000000, Generator Loss=1.606154
150: Discriminator Loss=0.000000, Generator Loss=1.484155
151: Discriminator Loss=0.000000, Generator Loss=2.602510
152: Discriminator Loss=0.000000, Generator Loss=1.539369
153: Discriminator Loss=0.000000, Generator Loss=2.465945
154: Discriminator Loss=0.000000, Generator Loss=2.296328
155: Discriminator Loss=0.000000, Generator Loss=1.833820
156: Discriminator Loss=0.000000, Generator Loss=1.792520
157: Discriminator Loss=0.000000, Generator Loss=1.956092
158: Discriminator Loss=0.000000, Generator Loss=2.747009
159: Discriminator Loss=0.000000, Generator Loss=1.619448
160: Discriminator Loss=0.000000, Generator Loss=1.614402
161: Discrimin

286: Discriminator Loss=0.000000, Generator Loss=0.435135
287: Discriminator Loss=0.000000, Generator Loss=0.434875
288: Discriminator Loss=0.000000, Generator Loss=0.434557
289: Discriminator Loss=0.000000, Generator Loss=0.434228
290: Discriminator Loss=0.000000, Generator Loss=0.571256
291: Discriminator Loss=0.000000, Generator Loss=0.436558
292: Discriminator Loss=0.000000, Generator Loss=0.433917
293: Discriminator Loss=0.000000, Generator Loss=0.863631
294: Discriminator Loss=0.000000, Generator Loss=0.434997
295: Discriminator Loss=0.000000, Generator Loss=0.442500
296: Discriminator Loss=0.000000, Generator Loss=0.433539
297: Discriminator Loss=0.000000, Generator Loss=0.433289
298: Discriminator Loss=0.000000, Generator Loss=0.499569
299: Discriminator Loss=0.000000, Generator Loss=0.472478
300: Discriminator Loss=0.000000, Generator Loss=0.474310
301: Discriminator Loss=0.000000, Generator Loss=0.467989
302: Discriminator Loss=0.000000, Generator Loss=0.508695
303: Discrimin

428: Discriminator Loss=0.000000, Generator Loss=1.390866
429: Discriminator Loss=0.000000, Generator Loss=1.127100
430: Discriminator Loss=0.000000, Generator Loss=1.337805
431: Discriminator Loss=0.000000, Generator Loss=1.250776
432: Discriminator Loss=0.000000, Generator Loss=1.377233
433: Discriminator Loss=0.000000, Generator Loss=1.237050
434: Discriminator Loss=0.000000, Generator Loss=1.159437
435: Discriminator Loss=0.000000, Generator Loss=1.415057
436: Discriminator Loss=0.000000, Generator Loss=1.047477
437: Discriminator Loss=0.000000, Generator Loss=1.351163
438: Discriminator Loss=0.000000, Generator Loss=1.703152
439: Discriminator Loss=0.000000, Generator Loss=1.319748
440: Discriminator Loss=0.000000, Generator Loss=1.156529
441: Discriminator Loss=0.000000, Generator Loss=1.337839
442: Discriminator Loss=0.000000, Generator Loss=2.107475
443: Discriminator Loss=0.000000, Generator Loss=1.060117
444: Discriminator Loss=0.000000, Generator Loss=1.373797
445: Discrimin

In [22]:
predicted_labels

tensor([1.3209e-35, 2.6756e-32, 4.0915e-33, 6.3469e-36, 5.8884e-36, 2.8191e-37,
        8.7052e-33, 1.2881e-34, 1.8578e-36, 5.8099e-38, 5.7483e-35, 4.0350e-31,
        0.0000e+00, 1.7862e-32, 0.0000e+00, 3.3405e-33, 1.4977e-34, 1.5072e-36,
        3.5092e-36, 1.3168e-34, 1.1718e-36, 1.3991e-33, 5.4338e-34, 1.2558e-32,
        8.1287e-38, 3.6659e-35, 1.0390e-31, 5.6724e-32, 1.4052e-31, 1.2814e-35,
        4.3028e-35, 3.8483e-37, 1.4474e-34, 6.3442e-32, 2.8646e-33, 4.6679e-36,
        1.2293e-31, 8.3854e-34, 2.1047e-33, 1.8853e-34, 2.4733e-35, 5.6418e-33,
        1.4056e-37, 1.0431e-35, 2.7901e-34, 1.8096e-31, 0.0000e+00, 5.3042e-35,
        1.0188e-33, 0.0000e+00, 1.0743e-34, 3.4736e-32, 7.0090e-33, 4.9967e-34,
        8.9107e-33, 2.8884e-33, 2.0725e-32, 1.0015e-28, 0.0000e+00, 1.4515e-36,
        1.3058e-34, 6.5979e-32, 8.8289e-37, 1.7503e-34, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+

In [8]:
test_noise = np.random.normal(0, 1, (1, 100))
test_noise = torch.Tensor(test_noise).to(device)

In [6]:
# test_x=torch.Tensor(np.random.normal(0, 1, (batch_size, 100)))
# class generator_layer1(nn.Module):
#     def __init__(self):
#         super(generator_layer1, self).__init__()
#         self.conv=nn.Sequential(
#             nn.Linear(100, 3136)
#         )
#     def forward(self, x):
#         x=self.conv(x)
#         return(x)
# generator(test_x)

In [7]:
# class generator_layer2(nn.Module):
#     def __init__(self, in_channel=64, out_channel=128, kernel_size=5, padding=2, batchnorm_momentum=0.1):
#         super(generator_layer2, self).__init__()
#        self.conv=nn.Sequential(
#        nn.Upsample(scale_factor=2),
#        nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, padding=padding),
#        nn.BatchNorm2d(out_channel, momentum=batchnorm_momentum),
#        get_activation()
#        )
#         self.conv1=nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        
#     make_layers(in_channel=64, out_channel=128, kernel_size=5, padding=2, batchnorm_momentum=0.1)
#     def forward(self, x):
#         x=x.view(-1, 64, 7, 7)
#         print(x.shape)
#         x=self.conv1(x)
#         return(x)